*Import libraries*

In [22]:
!pip install qiskit qiskit-aer -q

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer
from math import pi

In [34]:
def quantum_teleportation(theta, phi):
    qreg = QuantumRegister(3, "q")
    creg = ClassicalRegister(2, "c")
    qc = QuantumCircuit(qreg, creg)

    # Step 1: Prepare message qubit |ψ>
    qc.ry(theta, qreg[0])
    qc.rz(phi, qreg[0])

    # Step 2: Create Bell pair between q1 and q2
    qc.h(qreg[1])
    qc.cx(qreg[1], qreg[2])

    # Step 3: Bell measurement on q0 and q1
    qc.cx(qreg[0], qreg[1])
    qc.h(qreg[0])
    qc.measure(qreg[0], creg[0])
    qc.measure(qreg[1], creg[1])
    qc.barrier()

    # X correction (when classical = 01)
    x_circ = QuantumCircuit(1)
    x_circ.x(0)
    qc.if_test((creg, 1), x_circ, [qreg[2]], [])

    # Z correction (when classical = 10)
    z_circ = QuantumCircuit(1)
    z_circ.z(0)
    qc.if_test((creg, 2), z_circ, [qreg[2]], [])

    # XZ correction (when classical = 11)
    xz_circ = QuantumCircuit(1)
    xz_circ.x(0)
    xz_circ.z(0)
    qc.if_test((creg, 3), xz_circ, [qreg[2]], [])

    return qc

*Run test on simulator*

In [35]:
def test_teleport(theta, phi, label):
    qc = quantum_teleportation(theta, phi)
    sim = AerSimulator()
    result = sim.run(qc, shots=1000).result()
    counts = result.get_counts(qc)

    print(f"\n=== {label} ===")
    print("Measurement results:", counts)
    print("\nCircuit:")
    print(qc.draw(output="text"))

*Teleportation Tests*

In [36]:
tests = [
    ("|0>", 0, 0),
    ("|1>", pi, 0),
    ("|+>", pi/2, 0),
    ("|->", pi/2, pi),
    ("|i>", pi/2, pi/2),
]

for label, th, ph in tests:
    test_teleport(th, ph, label)


=== |0> ===
Measurement results: {'11': 240, '01': 258, '10': 259, '00': 243}

Circuit:
     ┌───────┐┌───────┐     ┌───┐┌─┐ ░                                     »
q_0: ┤ Ry(0) ├┤ Rz(0) ├──■──┤ H ├┤M├─░─────────────────────────────────────»
     └─┬───┬─┘└───────┘┌─┴─┐└┬─┬┘└╥┘ ░                                     »
q_1: ──┤ H ├──────■────┤ X ├─┤M├──╫──░─────────────────────────────────────»
       └───┘    ┌─┴─┐  └───┘ └╥┘  ║  ░ ┌────── ┌───┐ ───────┐ ┌────── ┌───┐»
q_2: ───────────┤ X ├─────────╫───╫──░─┤ If-0  ┤ X ├  End-0 ├─┤ If-0  ┤ Z ├»
                └───┘         ║   ║  ░ └──╥─── └───┘ ───────┘ └──╥─── └───┘»
                              ║   ║    ┌──╨──┐                ┌──╨──┐      »
c: 2/═════════════════════════╩═══╩════╡ 0x1 ╞════════════════╡ 0x2 ╞══════»
                              1   0    └─────┘                └─────┘      »
«                                           
«q_0: ──────────────────────────────────────
«                                           
«q_1: 